In [7]:
import pyodbc
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
desired_width = 320
pd.set_option('display.width', desired_width)


diem_2022_df = pd.read_csv('DATA_2022.csv', low_memory=False)

diem_2022_df = diem_2022_df.rename(
    columns={'sbd': 'SBD', 'ngu_van': 'Ngữ văn', 'toan': 'Toán', 'ngoai_ngu': 'Ngoại ngữ', 'vat_li': 'Vật lý',
             'hoa_hoc': 'Hoá học', 'sinh_hoc': 'Sinh học', 'lich_su': 'Lịch sử', 'dia_li': 'Địa lý', 'gdcd': 'GDCD'})
diem_2022_df['KHTN'] = round(
    (diem_2022_df['Hoá học'] + diem_2022_df['Vật lý'] + diem_2022_df['Sinh học']) / 3, 2)
diem_2022_df['KHXH'] = round(
    (diem_2022_df['Địa lý'] + diem_2022_df['Lịch sử'] + diem_2022_df['GDCD']) / 3, 2)

temp = {'1': 'TP HÀ NỘI', '2': 'TP HỒ CHÍ MINH', '3': 'TP HẢI PHÒNG', '4': 'TP ĐÀ NẴNG', '5': 'HÀ GIANG',
        '6': 'CAO BẰNG', '7': 'LAI CHÂU', '8': 'LÀO CAI', '9': 'TUYÊN QUANG', '10': 'LẠNG SƠN',
        '11': 'BẮC KẠN', '12': 'THÁI NGUYÊN', '13': 'YÊN BÁI', '14': 'SƠN LA', '15': 'PHÚ THỌ',
        '16': 'VĨNH PHÚC', '17': 'QUẢNG NINH', '18': 'BẮC GIANG', '19': 'BẮC NINH',
        '21': 'HẢI DƯƠNG', '22': 'HƯNG YÊN', '23': 'HÒA BÌNH', '24': 'HÀ NAM',
        '25': 'NAM ĐỊNH', '26': 'THÁI BÌNH', '27': 'NINH BÌNH', '28': 'THANH HÓA',
        '29': 'NGHỆ AN', '30': 'HÀ TĨNH',
        '31': 'QUẢNG BÌNH', '32': 'QUẢNG TRỊ', '33': 'THỪA THIÊN - HUẾ', '34': 'QUẢNG NAM',
        '35': 'QUẢNG NGÃI', '36': 'KON TUM', '37': 'BÌNH ĐỊNH', '38': 'GIA LAI',
        '39': 'PHÚ YÊN', '40': 'ĐẮK LẮK',
        '41': 'KHÁNH HÒA', '42': 'LÂM ĐỒNG', '43': 'BÌNH PHƯỚC', '44': 'BÌNH DƯƠNG',
        '45': 'NINH THUẬN', '46': 'TÂY NINH', '47': 'BÌNH THUẬN', '48': 'ĐỒNG NAI',
        '49': 'LONG AN', '50': 'ĐỒNG THÁP',
        '51': 'AN GIANG', '52': 'BÀ RỊA', '53': 'TIỀN GIANG', '54': 'KIÊN GIANG',
        '55': 'TP CẦN THƠ', '56': 'BẾN TRE', '57': 'VĨNH LONG', '58': 'TRÀ VINH', '59': 'SÓC TRĂNG',
        '60': 'BẠC LIÊU',
        '61': 'CÀ MAU', '62': 'ĐIỆN BIÊN', '63': 'ĐĂK NÔNG', '64': 'HẬU GIANG'}
location = []
location = np.floor(diem_2022_df['SBD'] // 10 ** 6)
location = location.astype(int)

tinhThanh = []
for i in range(len(location)):
    tmp = str(location[i])
    tinhThanh.append(temp.get(tmp))
diem_2022_df['Tỉnh thành'] = tinhThanh

diem_2022_df = pd.DataFrame({'SBD': diem_2022_df['SBD'], 'Tỉnh thành': diem_2022_df['Tỉnh thành'], 'Toán': diem_2022_df['Toán'],
                            'Ngữ văn': diem_2022_df['Ngữ văn'], 'Vật lý': diem_2022_df['Vật lý'], 'Hoá học': diem_2022_df['Hoá học'],
                             'Sinh học': diem_2022_df['Sinh học'], 'KHTN': diem_2022_df['KHTN'], 'Lịch sử': diem_2022_df['Lịch sử'],
                             'Địa lý': diem_2022_df['Địa lý'], 'GDCD': diem_2022_df['GDCD'], 'KHXH': diem_2022_df['KHXH'], 'Ngoại ngữ': diem_2022_df['Ngoại ngữ']})

In [20]:
conx2022 = pyodbc.connect('DRIVER={ODBC Driver 13 for SQL Server}; SERVER=NAM-S-LAPTOP; Database=BTLPython; UID=BTLPython; PWD=123456789;')
print(pyodbc.drivers())
cursor = conx2022.cursor()
column = ['SBD', 'Toán', 'Ngữ văn', 'Ngoại ngữ', 'Vật lý', 'Hoá học', 'Sinh học', 'Lịch sử', 'Địa lý', 'GDCD']
score = [0] * 10
for i in range(diem_2022_df['SBD'].count()):
    for j in range(len(column)):
        if np.isnan(diem_2022_df[column[j]][i]) == False:
            if column[j] == 'SBD':
                score[j] = int(diem_2022_df[column[j]][i])
            else:
                score[j] = float(diem_2022_df[column[j]][i])
        else:
            score[j] = 0
    cursor.execute(
        'INSERT INTO THPTQG2022(SBD, Toan, Ngu_van, Ngoai_ngu, Vat_ly, Hoa_hoc, Sinh_hoc, Lich_su, Dia_ly, GDCD)\
        VALUES(?, ?, ?, ?, ?, ?, ?, ?, ?, ?)', score[0], score[1], score[2], score[3], score[4], score[5], score[6], score[7], score[8], score[9]
    )

conx2022.commit()
conx2022.close()

['SQL Server', 'SQL Server Native Client 11.0', 'ODBC Driver 13 for SQL Server', 'SQL Server Native Client RDA 11.0', 'ODBC Driver 17 for SQL Server', 'ODBC Driver 18 for SQL Server']
